In [ ]:

# Sys imports
import json
import os
import sys
from tqdm import tqdm

os.environ["KERAS_BACKEND"] = "jax"

# Training API
import keras as keras
from keras import layers
import keras_nlp

# Data manipulation and exploration
import numpy as np
from scipy.stats import shapiro, kurtosis

# Data visualization
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

# NLP
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import jax.numpy as jnp

# MLOps API
import mlflow
mlflow.set_tracking_uri('http://localhost:5000')


# EDA

## Data loading
There are 72M titles and views in the data used.

In [ ]:
# DATA LOADER 
def parse_jsonl_optimized(filepath, num_lines_to_import=None):
    titles = []
    view_counts = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for i, line in enumerate(tqdm(file, desc="Processing")):
            # Check if the specified number of lines has been reached (if specified)
            if (num_lines_to_import is not None and i >= num_lines_to_import):
                break
            # Parse the current line
            data = json.loads(line)
            # Extract and store the title and view count
            titles.append(data['title'])
            view_counts.append(data['view_count'])
    return titles, view_counts

# Example usage: Import only the first 1000 lines from the file
file_path = '/mnt/datassd/train_data.jsonl'
num_lines_to_import = 50000  # You can adjust this number as needed
titles, view_counts = parse_jsonl_optimized(file_path, num_lines_to_import)

## Create $Log_{10}(Data)$ and create tokenizer

When views are analyzed on their own, the MSE loss function does not make a lot of sense, there is a lot of variation in the data.

What's why we take $log_{10}$ to get the order of magnitude of the views ($10^x$).

In [ ]:
y_view_count = np.where(np.log10(view_counts) == -np.inf, 0, np.log10(view_counts))

from transformers import AutoTokenizer
from concurrent.futures import ThreadPoolExecutor

# Load BERT tokanizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Encoding all 70M datapoints takes ~45mins

In [ ]:
""" SINGLE THREADED  """
encoded_inputs = [tokenizer.encode(title, add_special_tokens=True) for title in tqdm(titles, total=len(titles), desc="Encoding")]

In [ ]:
#""" SOMEWHAT MULTITHREADED """
#def encode_title(title):
#    return tokenizer.encode(title, add_special_tokens=True)
#with ThreadPoolExecutor() as executor:
#    encoded_inputs = list(tqdm(executor.map(encode_title, titles), total=len(titles), desc="Encoding"))

In the context of title engagement, UPPER CASED titles correlate with engagement, it is important to use a CASED tokenizer.

In [ ]:
# Testing cased inputs in tokenization
titulos_raros = [r'IS THIS TOKEN Cased?']
print(tokenizer.tokenize(titulos_raros[0]))
tokenizer.encode(titulos_raros[0], add_special_tokens=True)

## Data Normalization tests

First, let's see the distribution of view counts

In [ ]:
# See Y view count distribution
sns.histplot(y_view_count, kde=True)
plt.title("Log of View Count Distribution")
plt.xlabel("Log of View Count")
plt.ylabel("Frequency")
plt.show()

In [ ]:
shapiro_test_stat, shapiro_p_value = shapiro(y_view_count)
kurtosis_value = kurtosis(y_view_count, fisher=True)

shapiro_test_stat, shapiro_p_value, kurtosis_value

Now let's see the ditribution of lenghts of titles.

We need to have a cutoff at a certain token lenght. So let's visualize when it would be appropriate to cut titles off.

In [ ]:
sns.histplot([len(encoded_input) for encoded_input in encoded_inputs], bins=50)
plt.title("Histogram of tokenized title lengths")
plt.xlabel("Length of tokenized title")
plt.ylabel("Number of titles")
plt.show()

Probably at 40 tokens, the cutoff would preserve most of the information. (Remember YouTube titles have a 100 char limit)

In [ ]:
max_length = 40  # Ensure your data is adjusted accordingly

padded_inputs = pad_sequences(encoded_inputs, maxlen=max_length, padding='post', truncating='post')


In [ ]:
# Just making sure, all inputs are the same length of 40 
sns.histplot([len(padded_input) for padded_input in padded_inputs], bins=50)
plt.title("Histogram of padded title lengths")
plt.xlabel("Length of padded title")
plt.ylabel("Number of titles")
plt.show()

Some samples to see the PADs

In [ ]:
sample = [tokenizer.decode(padded_input) for padded_input in padded_inputs[90:100]]
sample

In [ ]:
# Define model architecture
vocab_size = 30522  # Adjusted to match BERT's vocabulary size for bert-base-cased
embedding_dim = 256
num_heads = 4
intermediate_dim = 512
transformer_encoder_layers = 3

X_t = jnp.array(padded_inputs)
Y_t = jnp.array(y_view_count)

inputs = keras.Input(shape=(max_length,), dtype='int32')
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=vocab_size, 
    sequence_length=max_length, 
    embedding_dim=embedding_dim,
)

x = embedding_layer(inputs)

for _ in range(transformer_encoder_layers):
    encoder = keras_nlp.layers.TransformerEncoder(
        num_heads=num_heads,
        intermediate_dim=intermediate_dim,
        activation='relu',
        dropout=0.1,
    )
    x = encoder(x)

x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(1, activation='relu')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

model.summary()

In [ ]:
class MlflowCallbackLogPerBatch(mlflow.keras_core.MLflowCallback):
    def on_batch_end(self, batch, logs=None):
        if self.log_every_n_steps is None or logs is None:
            return
        if (batch + 1) % self.log_every_n_steps == 0:
            self.metrics_logger.record_metrics(logs, self._log_step)
            self._log_step += self.log_every_n_steps


with mlflow.start_run() as run:
    model.fit(X_t, Y_t, batch_size=32, epochs=10, validation_split=0.2, callbacks= [MlflowCallbackLogPerBatch(run, log_every_epoch=False, log_every_n_steps=5)])
mlflow.end_run()

# Evaluation

In [ ]:
# Load the eval dataset
eval_file_path = '/mnt/datassd/eval_data.jsonl'
titles_eval, view_counts_eval = parse_jsonl_optimized(file_path, 1000)
encoded_evals = [tokenizer.encode(title, add_special_tokens=True) for title in tqdm(titles_eval, total=len(titles_eval), desc="Encoding")]
padded_evals = pad_sequences(encoded_evals, maxlen=max_length, padding='post', truncating='post')
view_evals = np.where(np.log10(view_counts_eval) == -np.inf, 0, np.log10(view_counts_eval))
X_e = jnp.array(padded_evals)
Y_e = jnp.array(view_evals)

In [ ]:
Y_pred = model.predict(X_e)

# Make scatter of predicted vs actual
plt.scatter(Y_e, Y_pred)
# Make a line
plt.plot([0, 10], [0, 10], color='red')
plt.title("Predicted vs Actual Log of View Count")
plt.xlabel("Actual Log of View Count")
plt.ylabel("Predicted Log of View Count")
plt.show()